In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
import spacy
import string
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

Import data set:

In [2]:
pd.options.mode.chained_assignment = None

col_list = ['text', 'class']
df_full = pd.read_csv('data/data_set.csv', usecols=col_list)
df = df_full[['text']]

df["text"] = df["text"].astype(str)
print(df_full.head())
print(df.head())

                                                text          class
0  "Μ.Βεργ. Έναν μήνα αφότου η Τουρκία αποχώρησε ...  Ανθρωποκτονία
1  "Μέχρι να αποκαλυφθεί η γυναικοκτονία της Καρο...  Γυναικοκτονία
2  "Γεμάτο υποθέσεις που συγκλόνισαν την κοινή γν...  Γυναικοκτονία
3  "Στον ανακριτή οδηγούνται σήμερα οι δράστες τη...  Ανθρωποκτονία
4  "Συνέβη ξανά. Την περασμένη εβδομάδα ένα σπορ ...  Ανθρωποκτονία
                                                text
0  "Μ.Βεργ. Έναν μήνα αφότου η Τουρκία αποχώρησε ...
1  "Μέχρι να αποκαλυφθεί η γυναικοκτονία της Καρο...
2  "Γεμάτο υποθέσεις που συγκλόνισαν την κοινή γν...
3  "Στον ανακριτή οδηγούνται σήμερα οι δράστες τη...
4  "Συνέβη ξανά. Την περασμένη εβδομάδα ένα σπορ ...


## Text preprocessing:

lower casing:

In [3]:

df["text_lower"] = df["text"].str.lower()
print(df.head())
# drop the new column created in last cell
df.drop(["text_lower"], axis=1, inplace=True)


                                                text  \
0  "Μ.Βεργ. Έναν μήνα αφότου η Τουρκία αποχώρησε ...   
1  "Μέχρι να αποκαλυφθεί η γυναικοκτονία της Καρο...   
2  "Γεμάτο υποθέσεις που συγκλόνισαν την κοινή γν...   
3  "Στον ανακριτή οδηγούνται σήμερα οι δράστες τη...   
4  "Συνέβη ξανά. Την περασμένη εβδομάδα ένα σπορ ...   

                                          text_lower  
0  "μ.βεργ. έναν μήνα αφότου η τουρκία αποχώρησε ...  
1  "μέχρι να αποκαλυφθεί η γυναικοκτονία της καρο...  
2  "γεμάτο υποθέσεις που συγκλόνισαν την κοινή γν...  
3  "στον ανακριτή οδηγούνται σήμερα οι δράστες τη...  
4  "συνέβη ξανά. την περασμένη εβδομάδα ένα σπορ ...  


remove punctuation

In [4]:

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["text_wo_punct"] = df["text"].apply(lambda text: remove_punctuation(text))
print(df.head())

                                                text  \
0  "Μ.Βεργ. Έναν μήνα αφότου η Τουρκία αποχώρησε ...   
1  "Μέχρι να αποκαλυφθεί η γυναικοκτονία της Καρο...   
2  "Γεμάτο υποθέσεις που συγκλόνισαν την κοινή γν...   
3  "Στον ανακριτή οδηγούνται σήμερα οι δράστες τη...   
4  "Συνέβη ξανά. Την περασμένη εβδομάδα ένα σπορ ...   

                                       text_wo_punct  
0  ΜΒεργ Έναν μήνα αφότου η Τουρκία αποχώρησε από...  
1  Μέχρι να αποκαλυφθεί η γυναικοκτονία της Καρολ...  
2  Γεμάτο υποθέσεις που συγκλόνισαν την κοινή γνώ...  
3  Στον ανακριτή οδηγούνται σήμερα οι δράστες της...  
4  Συνέβη ξανά Την περασμένη εβδομάδα ένα σπορ αυ...  


ML :-

In [ ]:
# class_distribution = (df['class'].value_counts() * 100) / 346
# plt.bar(class_distribution.index, class_distribution)
# plt.show()

# print(class_distribution)

# See missing values:
# print(df.isna().sum())


# Split data set to train and test
# training_data, testing_data = train_test_split(df, test_size=0.2, random_state=25)
# print(f"No. of training examples: {training_data.shape[0]}")
# print(f"No. of testing examples: {testing_data.shape[0]}")

# stop_words = stopwords.words('greek')


# TODO: split train data set test validation set: Idecide whether this will happen on the train data set)

# X=training_data['text']
# y=training_data['class']

# X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state=1)
# kf = KFold(n_splits=2)
# kf.get_n_splits(X_train)
# for train_index, test_index in kf.split(X):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
